In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from domoku.constants import BLACK, WHITE
from domoku.tools import GomokuTools as Gt

In [3]:
input_size=7

---

## Sampling Random Boards

In [4]:
from notebooks.ml_basics_recap.data import new_sample
sample = new_sample(board_size=input_size, num_blacks=20, num_whites=0)
sample.shape

(1, 7, 7, 2)

In [5]:
Gt.print_bin(sample, True)

shape: (7, 7, 2)
[[0. 1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1.]]


---

## The Detection Map

In [6]:
from notebooks.ml_basics_recap.models.heuristic_detector import HeuristicDetector
detector = HeuristicDetector(input_size)

2022-05-26 20:38:38.597174: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tf.floor(detector(sample)+.01)

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

---

## Batches From a Dataset

In [8]:
BATCH_SIZE=2

In [9]:
from notebooks.ml_basics_recap.data import new_sample, new_dataset
dataset = new_dataset(100, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)

In [10]:
iterator  = iter(dataset)
states, labels = iterator.next()
labels.shape

TensorShape([2, 1, 7, 7])

In [11]:
states.shape

TensorShape([2, 1, 7, 7, 2])

---

## The Trainable Model

In [12]:
from notebooks.ml_basics_recap.models import SimpleConvQFunction

model_q = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)

In [29]:
print(model_q(states).shape)
model_q(states).shape

(1, 1, 7, 7)


TensorShape([1, 1, 7, 7])

---

## Training

In [14]:
TRAIN_SIZE = 1024 * 16
TEST_SIZE = 1024
BATCH_SIZE = 512

In [15]:
train_dataset = new_dataset(size=TRAIN_SIZE,
                            sampler=lambda: new_sample(board_size=input_size, num_blacks=10, num_whites=0),
                            labeler=detector
                            ).batch(BATCH_SIZE)
test_dataset = new_dataset(size=TEST_SIZE,
                           sampler=lambda: new_sample(board_size=input_size, num_blacks=10, num_whites=0),
                           labeler=detector
                           ).batch(BATCH_SIZE)

In [16]:
from notebooks.ml_basics_recap.training import Trainer

trainer = Trainer(model_q, train_data=train_dataset, test_data=test_dataset)
trainer.train(20)

Epoch 1, Loss: 0.010557678528130054, Accuracy: 0.10275056213140488,     Test Loss: 0.008081444539129734, Test Accuracy: 0.08989685773849487
Epoch 2, Loss: 0.005673340987414122, Accuracy: 0.07532157748937607,     Test Loss: 0.0031691845506429672, Test Accuracy: 0.05629551410675049
Epoch 3, Loss: 0.0018852961948141456, Accuracy: 0.04341999813914299,     Test Loss: 0.0007818794692866504, Test Accuracy: 0.027962109073996544
Epoch 4, Loss: 0.0004476092872209847, Accuracy: 0.021156778559088707,     Test Loss: 0.0006987829692661762, Test Accuracy: 0.026434503495693207
Epoch 5, Loss: 0.00023125337611418217, Accuracy: 0.015207016840577126,     Test Loss: 0.00014985751477070153, Test Accuracy: 0.012241629883646965
Epoch 6, Loss: 5.966650860500522e-05, Accuracy: 0.007724409457296133,     Test Loss: 5.678702291334048e-05, Test Accuracy: 0.007535716984421015
Epoch 7, Loss: 2.874143137887586e-05, Accuracy: 0.0053611034527421,     Test Loss: 4.5684337237617e-05, Test Accuracy: 0.006759018171578646
Ep

---

## Manual Inspection

In [30]:
BATCH_SIZE=1

In [31]:
dataset = new_dataset(10, lambda: new_sample(input_size, 10, 0), detector).batch(BATCH_SIZE)

In [32]:
iterator  = iter(dataset)
states, labels = iterator.next()

### Labels

In [34]:
tf.floor(labels+.01)

<tf.Tensor: shape=(1, 1, 7, 7), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]]], dtype=float32)>

### Learned

In [35]:
pred = model_q(states)
np.round(pred, 2)

array([[[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [-0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0., -0.,  0.,  0.],
         [-0.,  0.,  0.,  0.,  0.,  0., -0.]]]], dtype=float32)